In [2]:
# 필수 라이브러리 불러오기
import os
import openai
from dotenv import load_dotenv
from typing import List, Tuple

# .env 파일에서 OPENAI API 키 로드
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# 대화 기록 저장용 리스트 초기화
chat_history = []  # [{'user': ..., 'bot': ...}, ...] 형태

In [4]:
# MBTI 유형별 말투 스타일 정의
MBTI_TONE = {
    "ENFP": "따뜻하고 유쾌하며 이모티콘을 자주 사용합니다.",
    "ISTJ": "분석적이고 신중하며 단정한 말투입니다.",
    "INFP": "섬세하고 감정에 공감하는 부드러운 말투입니다.",
    "ESTJ": "단호하고 체계적이며 사실 위주의 말투입니다.",
    "INTP": "논리적이고 중립적인 말투입니다.",
    "ESFJ": "친근하고 배려심 많은 말투로 위로를 잘 전합니다.",
    "ENTP": "재치 있고 유머러스하며 아이디어를 자유롭게 표현합니다.",
    "ISFJ": "조용하지만 따뜻하고 배려 깊은 말투로 상대를 존중합니다.",
    "INFJ": "직관적이며 깊이 있는 표현과 따뜻한 공감이 어우러진 말투입니다.",
    "ESTP": "직설적이고 에너지 넘치며 상황 중심적으로 조언합니다.",
    "ISFP": "차분하고 부드러우며 감정에 민감하게 반응합니다.",
    "INTJ": "간결하고 직관적인 말투이며 효율 중심적으로 접근합니다.",
    "ENTJ": "자신감 있고 목표 지향적이며 명확한 표현을 사용합니다.",
    "ENFJ": "따뜻하고 포용적인 말투로 감정에 깊이 공감합니다.",
    "ISTP": "과묵하고 실용적인 조언 위주로 핵심만 전달합니다.",
    "ESFP": "밝고 생동감 있는 말투로 친근하고 즉흥적인 표현을 자주 사용합니다."
}

# MBTI 궁합형 매핑 (서로 잘 맞는 성향 기준)
MBTI_COMPATIBILITY_MAP = {
    "ISTJ": "ISFJ",
    "ISFJ": "ISTJ",
    "INFJ": "ENFP",
    "INTJ": "ENFP",
    "ISTP": "ESTP",
    "ISFP": "ESFP",
    "INFP": "ENFJ",
    "INTP": "ENTP",
    "ESTP": "ISFP",
    "ESFP": "ISFP",
    "ENFP": "INFJ",
    "ENTP": "INFJ",
    "ESTJ": "ESFJ",
    "ESFJ": "ISFJ",
    "ENFJ": "INFP",
    "ENTJ": "INTP"
}

# 상담 주제별 주요 키워드 정의
COUNSELING_TOPICS = {
    "전문상담": ["우울증", "불면증", "약물", "자해", "자살", "정신과", "PTSD", "조현병", "공황"],
    "진로상담": ["진로", "이직", "취업", "자소서", "면접", "전공", "직업"],
    "관계상담": ["연애", "이별", "가족", "친구", "갈등", "불화", "인간관계"],
    "학업상담": ["성적", "시험", "학점", "졸업", "공부", "지각"],
    "자기이해": ["자존감", "자책", "후회", "무기력", "자기혐오", "의욕 없음"],
    "라이프스타일": ["여행", "계획", "루틴", "취미", "쉬고 싶어", "생일", "놀러", "추천", "휴가"],
    "경제고민": ["돈", "알바", "월세", "생활비", "경제적 어려움", "빚", "소득", "용돈"],
    "결정갈등": ["선택", "결정", "포기할까", "고민 중", "우선순위", "갈등 중", "결단"],
    "자기계발": ["루틴", "습관", "목표", "계획", "성장하고 싶어", "나아지고 싶어"],
    "건강관리": ["운동", "체력", "수면", "식습관", "건강", "몸"],
    "디지털피로": ["인스타", "틱톡", "SNS", "휴대폰", "유튜브", "중독", "정보 과부하"],
    "미래불안": ["불확실", "미래", "막막함", "예측할 수 없음", "불안"],
    "퇴사고민": ["퇴사", "일하기 싫어", "번아웃", "사직", "이직 고민", "회피"]
}

In [5]:
# 사용자의 입력이 정보 탐색인지, 감정 상담인지 구분
def detect_intent(user_input: str) -> str:
    """
    사용자의 입력 문장에서 정보요청 여부를 판단합니다.
    특정 키워드가 포함되어 있으면 정보요청으로 간주합니다.
    """
    info_keywords = ["알려줘", "무엇", "왜", "정보", "정리", "통계", "수치", "근거"]
    return "정보요청" if any(kw in user_input for kw in info_keywords) else "감정상담"


# 입력된 문장에서 주요 상담 주제를 추정
def detect_topic(user_input: str) -> str:
    """
    상담 주제 사전에 따라 사용자의 입력 내용을 기반으로
    적절한 상담 카테고리를 반환합니다.
    """
    for topic, keywords in COUNSELING_TOPICS.items():
        if any(kw in user_input for kw in keywords):
            return topic
    return "일반"


# MBTI의 T/F 성향 추출 (감정형/사고형 구분)
def get_tf_trait(mbti: str) -> str:
    """
    MBTI 문자열에서 3번째 글자를 기준으로 T/F 성향을 반환합니다.
    F이면 감정형, T이면 사고형입니다.
    """
    return "F" if mbti[2].upper() == "F" else "T"

In [6]:
# 사용자의 MBTI에서 말투 스타일과 궁합형을 찾아 반환
def get_mbti_style_and_compatibility(mbti_input: str) -> Tuple[str, str]:
    """
    입력된 MBTI 문자열을 기준으로:
    - 상담에 사용할 말투 스타일 (궁합형 기준)
    - 궁합이 잘 맞는 MBTI 유형
    을 반환합니다.
    """
    mbti = mbti_input.strip().upper()

    if mbti not in MBTI_TONE:
        raise ValueError(f"올바르지 않은 MBTI 유형입니다: {mbti}")

    compatible_mbti = MBTI_COMPATIBILITY_MAP.get(mbti, "ENFP")  # 기본값은 ENFP
    tone = MBTI_TONE.get(compatible_mbti, "따뜻하고 부드러운 말투입니다.")

    return tone, compatible_mbti

In [7]:
def generate_counseling_response(user_input: str, predicted_mbti: str) -> str:
    """
    사용자의 MBTI 및 입력 내용에 기반해 적절한 상담 응답을 생성합니다.
    - MBTI에 따른 말투 스타일 적용
    - 상담 주제에 따라 응답 문체 보정
    - 감정형/사고형에 따라 공감 중심 or 논리 중심 조언
    """

    # 말투 스타일과 궁합형 MBTI 추출
    tone, compatible_mbti = get_mbti_style_and_compatibility(predicted_mbti)
    
    # 감정형(F) 또는 사고형(T) 분기
    tf_trait = get_tf_trait(predicted_mbti)
    
    # 상담 주제 추정 (예: 진로상담, 자기이해 등)
    topic = detect_topic(user_input)

    # 전문 상담은 별도 응답으로 처리
    if topic == "전문상담":
        return (
            "이 주제는 전문가의 도움이 가장 안전하고 효과적일 수 있어요.\n"
            "정신건강복지센터 또는 의료기관 상담을 권장드립니다.\n"
            "https://www.mentalhealth.or.kr/"
        )

    # 기본 응답 스타일 설정
    base_instruction = (
        "감정에 공감하며 부드럽게 위로해주세요." if tf_trait == "F"
        else "논리적이고 분석적으로 조언해주세요."
    )

    # 주제별 추가 스타일 조정
    topic_style = {
        "자기이해": " 자기 수용과 회복을 도와주는 말이 포함되면 좋습니다.",
        "진로상담": " 현실적인 조언과 격려가 있으면 좋습니다.",
        "관계상담": " 인간관계의 감정적 복잡함을 이해해주세요.",
        "학업상담": " 스트레스를 덜어줄 수 있는 따뜻한 조언이면 좋습니다.",
        "경제고민": " 경제적 상황에 대한 공감과 실질적인 팁이 있으면 좋아요."
    }

    # 최종 지시문 생성
    instruction = base_instruction + topic_style.get(topic, "")

    # GPT 대화 메시지 구성
    messages = [
        {
            "role": "system",
            "content": (
                f"{compatible_mbti} 유형의 말투를 사용하는 심리상담 챗봇입니다.\n"
                f"말투 특징: {tone} {instruction}"
            )
        },
        {"role": "user", "content": user_input}
    ]

    # 이전 대화 이력 삽입
    for turn in chat_history:
        messages.insert(1, {"role": "assistant", "content": turn["bot"]})
        messages.insert(1, {"role": "user", "content": turn["user"]})

    # GPT 응답 생성
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7,
        max_tokens=400
    )

    # 결과 저장 및 반환
    reply = response.choices[0].message["content"].strip()
    append_history(user_input, reply)
    return reply

In [8]:
# 정보성 질문에 대한 요약 응답 생성 함수 (출처 URL 포함)
def search_and_summarize_with_sources(query: str) -> str:
    """
    사용자의 정보 요청에 대해 웹 검색 결과를 요약하고,
    관련 출처 링크를 함께 반환합니다.
    
    현재는 예시용 더미 데이터를 사용합니다.
    실제 적용 시에는 Bing API 또는 SERP API 연동이 필요합니다.
    """

    # 예시용 더미 검색 결과 (title, url 포함)
    sources = [
        {"title": "2025 최저임금 공고", "url": "https://www.moel.go.kr/news/2025wage"},
        {"title": "최저임금 계산기", "url": "https://nodong.kr/MinimumWage2025"},
        {"title": "최저임금 인상 요약", "url": "https://flex.blog/2025minwage"}
    ]

    # 요약 본문 (실제 검색 요약 결과로 대체 가능)
    summary = (
        "2025년 최저임금은 시간당 10,030원이며 전년 대비 1.7% 인상되었습니다.\n"
        "주 40시간 기준으로 월 약 2,096,270원에 해당합니다.\n\n"
        "참고 출처:\n" +
        "\n".join([f"- {item['title']}: {item['url']}" for item in sources])
    )

    return summary

In [9]:
# 사용자 입력과 챗봇 응답을 히스토리에 추가
def append_history(user_input: str, bot_output: str):
    """
    대화 흐름 유지를 위해 최근 몇 회의 대화를 저장합니다.
    - 최대 저장 개수를 초과하면 오래된 기록을 제거합니다.
    """
    chat_history.append({"user": user_input, "bot": bot_output})
    
    # 대화 기록이 너무 길어지지 않도록 최대 6개까지만 유지
    if len(chat_history) > 6:
        chat_history.pop(0)

In [10]:
# 사용자 입력을 받아 목적에 따라 분기하고 응답 생성
def run_chatbot(user_input: str, mbti: str):
    """
    사용자 입력과 MBTI에 기반하여,
    - 감정상담 또는
    - 정보요청
    으로 분기한 뒤, 적절한 응답을 출력합니다.
    """

    # 1. 입력 목적 판단 (정보요청 vs 감정상담)
    intent = detect_intent(user_input)

    # 2. 감정상담이면 상담 응답 생성
    if intent == "감정상담":
        response = generate_counseling_response(user_input, mbti)
    
    # 3. 정보요청이면 검색 + 요약 + 출처 포함 응답 생성
    else:
        response = search_and_summarize_with_sources(user_input)
    
    return response

In [13]:
# 확장된 감정 키워드 목록
emotion_bank = [
    # 부정 감정
    "우울", "불안", "무기력", "외로움", "자기혐오", "혼란", "자책", "답답함", "막막함",
    "지침", "슬픔", "공허함", "무의미함", "분노", "억울함", "후회", "질투", "두려움",
    "실망", "죄책감", "초조함", "허탈함", "피로", "지루함", "소외감",

    # 긍정 감정
    "기대", "희망", "설렘", "감사함", "안도감", "성취감", "행복", "흥미", "자신감",
    "든든함", "충만함", "유쾌함", "사랑받는 느낌", "의미 있음", "뿌듯함", "기분 좋음",

    # 복합/혼합 감정
    "불안하지만 기대됨", "설레지만 걱정됨", "외롭지만 자유로움", "두렵지만 도전하고 싶음",
    "불안과 희망이 함께 있음"
]

# 감정 키워드 추출 함수
def extract_emotion_keywords(user_input: str) -> List[str]:
    """
    사용자 입력에서 주요 감정 키워드를 추출합니다.
    확장된 감정 키워드 리스트에 기반해 매칭합니다.
    """
    return [emotion for emotion in emotion_bank if emotion in user_input]


# 감정 기반 음악 추천 함수
def recommend_music(emotions: List[str]) -> List[str]:
    """
    감정 키워드에 따라 어울리는 음악을 추천합니다.
    간단한 매핑 기반이며, 향후 검색 기반 추천으로 확장 가능
    """
    if not emotions:
        return []

    mapping = {
        "무기력": ["봄날 - BTS", "Breathe - 이하이", "Way Back Home - Shaun"],
        "불안": ["밤편지 - 아이유", "Way Back Home - Shaun"],
        "우울": ["그대라는 시 - 태연", "Empty - WINNER"],
        "외로움": ["혼자 - 거미", "Lonely - 종현"],
        "기대": ["기다릴게 - 하동균", "너를 만나 - 폴킴"],
        "설렘": ["Love Scenario - iKON", "좋아 - 민서"],
        "자신감": ["I Am - IVE", "Not Shy - ITZY"],
        "희망": ["소우주 - BTS", "꿈을 꾼다 - 양파"],
        "두려움": ["겁 - 송민호", "공포 - 노을"]
    }

    result = []
    for emotion in emotions:
        if emotion in mapping:
            result.extend(mapping[emotion])
    return list(set(result))


# 감정 이미지 묘사 생성 함수 (GPT 호출)
def generate_emotion_visual(emotions: List[str]) -> str:
    """
    감정을 기반으로 GPT를 통해 시적 감정 묘사 텍스트를 생성합니다.
    """
    if not emotions:
        return "따뜻한 햇살이 스며드는 조용한 방처럼 평온한 감정이에요."

    prompt = f"{', '.join(emotions)} 감정을 풍경이나 장면으로 시적으로 묘사해주세요."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "당신은 감정을 시적으로 시각화하는 묘사 작가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=100
    )

    return response.choices[0].message["content"].strip()


# 통합된 챗봇 실행 함수 (감정 분석 포함)
def run_chatbot(user_input: str, mbti: str) -> dict:
    """
    전체 챗봇 실행 흐름.
    입력에 따라 감정상담/정보요청을 분기하고,
    감정 키워드, 음악 추천, 이미지 묘사를 포함한 결과를 반환합니다.
    """
    intent = detect_intent(user_input)

    if intent == "정보요청":
        response = search_and_summarize_with_sources(user_input)
        return {
            "intent": "정보요청",
            "response": response,
            "emotions": [],
            "songs": [],
            "visual": None
        }

    else:
        response = generate_counseling_response(user_input, mbti)
        emotions = extract_emotion_keywords(user_input)
        songs = recommend_music(emotions)
        visual = generate_emotion_visual(emotions)

        return {
            "intent": "감정상담",
            "response": response,
            "emotions": emotions,
            "songs": songs,
            "visual": visual
        }